# Overview

We build our own model in [Create simple model using PyTorch](https://www.kaggle.com/code/aisuko/create-simple-model-using-pytorch), and we preared data in [Datasets and Dataloaders in PyTorch](https://www.kaggle.com/code/aisuko/datasets-and-dataloaders-in-pytorch). It's time to train, validate and test our model by optimizing its parameters on our data. 

Training a model is an iterative process in each iteration the model makes a guess about the output, calculates the error in its guess(loss), collects the derivatives of the error with respect to its parameters, and **optimizes** these parameters using gradient descent.

# Loading the dataset

We use the dataset from `torchvision`.

In [1]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data=datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data=datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader=DataLoader(training_data, batch_size=64)
test_dataloader=DataLoader(test_data, batch_size=64)

100%|██████████| 26421880/26421880 [00:02<00:00, 10097683.24it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 206003.78it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3837832.92it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 13648721.23it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [2]:
len(train_dataloader)

938

# Define the model

In [3]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

model=NeuralNetwork()

# Hyperparameters

Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates.

We define the following hyperparameters for training:

* **Number of Epochs** - the number times to iterate over the dataset.
* **Batch Size** -  the number of data samples propagated through the network before the parameters are updated
* **Learning Rate** - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [4]:
learning_rate=1e-3
batch_size=64
epochs=5

# Training(Optimization Loop)

Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an **epoch**.

Each epoch consists of two main parts:
* **The Train Loop** -  iterate over the training dataset and try to converge to optimal parameters.
* **The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.


## Loss Function

When presented with some training data, our untrained network is likely not to give the correct answer. **Loss function** measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include `nn.MSELoss`(Mean Square Error) for regression tasks, and `nn.NLLLoss`(Negative Log Likelihood) for classification. `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`.

We pass our model's output logits to `nn.CrossEntropyLoss`, which will normalize the logits and compute the prediction error.

In [5]:
# initialize the loss function
loss_fn=nn.CrossEntropyLoss()

## Optimizer

Optimization is the process of adjusting model parameters to reduce model error in each training step. **Optimization algorithms** define how this process is performed(here we use Stochastic Gradient Descent). All optimization logic is encapsulated in the `optimizer` object. Here, we use SGD optimizer; additionally, there are many of others avaliable in PyTorch like:
* ADAM
* RMSProp

We initialize the optimizer by registering the model's parameters that need to be trained, and passing in the learning rate hyperparameter.

In [6]:
import torch
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:
* Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up;to prevent double-counting we explicity zero them at each iteration.
* Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.
* Once we have our gradients, we call `optimizer.step()` to adjust the paeameters by the gradients collected in the backward pass.

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        # compute prediction and loss
        pred=model(X)
        loss=loss_fn(pred, y)
        
        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch %100==0:
            loss,current=loss.item(), batch*batch_size+len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')
        